In [1]:
#Assignment 4 Solutions

In [2]:
#Import libraries
library(bigrquery)
library(dplyr)
library(DBI)


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union




In [3]:
#Establish connection with big query baseball dataset
con <- dbConnect(
    bigrquery::bigquery(),
    project="bigquery-public-data",
    dataset="baseball",
    billing="upheld-modem-331422")

In [4]:
#List database tables as connection test
dbListTables(con)

[1] "games_post_wide" "games_wide"      "schedules"

In [5]:
#Question 1
dbGetQuery(con, "SELECT dayNight, avg(duration_minutes) as avg_duration FROM bigquery-public-data.baseball.schedules 
GROUP BY dayNight")

dayNight,avg_duration
<chr>,<dbl>
D,184.9798
N,184.5040


In [7]:
#Question 2
dbGetQuery(con, "SELECT homeTeamName, avg(attendance) as avg_attendance FROM bigquery-public-data.baseball.schedules 
WHERE homeTeamName != 'American League' GROUP BY homeTeamName ORDER BY avg_attendance DESC")

homeTeamName,avg_attendance
<chr>,<dbl>
Dodgers,45719.90
Cardinals,42524.57
Blue Jays,41880.36
Giants,41550.81
Cubs,39906.42
Yankees,37801.91
Angels,37236.30
Red Sox,36486.84
Mets,34900.22


In [9]:
#Question 3
dbGetQuery(con, "SELECT hitterId, hitterFirstName, hitterLastName, count(*) AS HR FROM bigquery-public-data.baseball.games_wide 
WHERE outcomeDescription='Homerun' GROUP BY hitterId, hitterFirstName, hitterLastName ORDER BY HR DESC LIMIT 5")

hitterId,hitterFirstName,hitterLastName,HR
<chr>,<chr>,<chr>,<int>
aff8ce97-4a84-43db-bdaf-3517b6b5fa88,Mark,Trumbo,47
01eaff59-9b34-47fd-af64-54b2e6f22628,Nelson,Cruz,44
8a3eae26-3aa7-42fb-92c3-273f5004e4f1,Khris,Davis,42
8ec56596-5b8b-41f8-88a7-384f20b8b6a7,Edwin,Encarnacion,42
29a80d91-946d-4701-af7d-034850bdef00,James,Dozier,42


In [10]:
#Question 4

#Part A
reg_season <- dbGetQuery(con, "SELECT DISTINCT(gameId), homeTeamName, awayTeamName, homeFinalRuns, awayFinalRuns 
FROM bigquery-public-data.baseball.games_wide WHERE homeTeamName='Blue Jays' OR awayTeamName='Blue Jays'")
head(reg_season)

gameId,homeTeamName,awayTeamName,homeFinalRuns,awayFinalRuns
<chr>,<chr>,<chr>,<int>,<int>
252e4911-b809-48b1-8a53-6fcfd1f4c26c,Rays,Blue Jays,8,3
50e37133-01e3-4080-a6b2-4f08e79c160f,Rays,Blue Jays,7,5
94dc70b6-3018-46ab-8775-defa223fcefa,Rays,Blue Jays,3,5
e32d51d3-ac70-41fc-8963-7adc012fef8d,Rangers,Blue Jays,7,6
223768a6-a3b0-4a47-ba0e-5c64f8df7d59,Blue Jays,Angels,2,8
6827fa1a-7708-4cb6-bdc2-d697d84c31f4,Blue Jays,Red Sox,3,13


In [12]:
#Part B
post_season <- dbGetQuery(con, "SELECT DISTINCT(gameId), homeTeamName, awayTeamName, homeFinalRuns, awayFinalRuns 
FROM bigquery-public-data.baseball.games_post_wide WHERE homeTeamName='Blue Jays' OR awayTeamName='Blue Jays'")
head(post_season)

gameId,homeTeamName,awayTeamName,homeFinalRuns,awayFinalRuns
<chr>,<chr>,<chr>,<int>,<int>
04a6af00-7df7-49d0-9bd0-1ffbb76b8e22,Rangers,Blue Jays,1,10
4ad2de69-467a-4755-bad1-2362425ca6dd,Indians,Blue Jays,2,1
4da8ee8a-ea2c-4625-8ef4-584a5302cf24,Indians,Blue Jays,2,0
50599fd0-e5a8-4330-b185-cf99db1f5b89,Blue Jays,Indians,5,1
6d62f75d-2021-46d8-8c8c-f27006914cff,Blue Jays,Rangers,7,6
909aaaf2-63c7-4a47-8c01-24fd06a9a7bb,Blue Jays,Indians,2,4


In [14]:
#Part C
blue_jays_combined <- rbind(reg_season, post_season)

#Create home binary helper column
blue_jays_combined$Home <- ifelse(blue_jays_combined$homeTeamName=="Blue Jays", 1, 0)

#Create win column
blue_jays_combined$Win <- ifelse((blue_jays_combined$Home==1 & 
                                  (blue_jays_combined$homeFinalRuns > blue_jays_combined$awayFinalRuns)) | 
                                 (blue_jays_combined$Home==0 & 
                                  (blue_jays_combined$homeFinalRuns < blue_jays_combined$awayFinalRuns)), 1, 0)
#Preview new col
head(blue_jays_combined)

gameId,homeTeamName,awayTeamName,homeFinalRuns,awayFinalRuns,Home,Win
<chr>,<chr>,<chr>,<int>,<int>,<dbl>,<dbl>
252e4911-b809-48b1-8a53-6fcfd1f4c26c,Rays,Blue Jays,8,3,0,0
50e37133-01e3-4080-a6b2-4f08e79c160f,Rays,Blue Jays,7,5,0,0
94dc70b6-3018-46ab-8775-defa223fcefa,Rays,Blue Jays,3,5,0,1
e32d51d3-ac70-41fc-8963-7adc012fef8d,Rangers,Blue Jays,7,6,0,0
223768a6-a3b0-4a47-ba0e-5c64f8df7d59,Blue Jays,Angels,2,8,1,0
6827fa1a-7708-4cb6-bdc2-d697d84c31f4,Blue Jays,Red Sox,3,13,1,0


In [15]:
#Part C - Continued
sum(blue_jays_combined$Win)

[1] 94

In [17]:
#Question 5

#Part A
kershaw_arsenal <- dbGetQuery(con, "SELECT pitchType, COUNT(*) as Num_Pitches FROM bigquery-public-data.baseball.games_wide 
WHERE pitcherLastName='Kershaw' AND pitcherFirstName='Clayton' GROUP BY pitchType")
kershaw_arsenal

pitchType,Num_Pitches
<chr>,<int>
FA,1049
SL,679
CU,321
,43
IB,4
CH,9
CT,1


In [25]:
#Part B
kershaw_final <- kershaw_arsenal[-4, ]
kershaw_final$Percentage <- round((kershaw_final$Num_Pitches/sum(kershaw_final$Num_Pitches)), 3)
kershaw_final

pitchType,Num_Pitches,Percentage
<chr>,<int>,<dbl>
FA,1049,0.508
SL,679,0.329
CU,321,0.156
IB,4,0.002
CH,9,0.004
CT,1,0.000


In [ ]:
#Question 6

#SQL, also known as structured query language, is the language used to interact with relational databases. 
#DBMS's, on the other hand, are wholistic systems used to interact with relational databases. The most common 
#DBMS's like MySQL, PostgreSQL, and SQL Server all use the SQL language.

In [ ]:
#Question 7

#Joins are used to combine data columnwise (i.e. adding new columns to a table), while unions are used to combine data rowwise. 
#Joins are similar to VLOOKUP in Excel while unions are analagous to extending an Excel table by adding new rows to it. 
#Joins require at least one common column between the tables while unions require all columns in both tables to be the same.